# Dimension Reduction Plots

This notebook demonstrates how the plotting functions in ScLive can be used with AnnData objects.

First we need to create an AnnData object with some annotations. The fastest way is to run scanpy pipeline with all the defaults (https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html).

The following does exactly this. The code assumes that there is genomics data in a folder named test_data/pbmc in the root folder. The data can be downloaded from 10xgenomics website [here](https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz). It saves the resulting annotated data to an output file. This code doesn't run all the code in the related tutorial and it doesn't necessarily provide nice results as it serves as an example dataset. It also adds some dummy annotations for plotting purposes.

In [ ]:
import os
import numpy as np
import scanpy as sc
import anndata

In [ ]:
INPUT_PATH = "../../test_data/pbmc"
OUTPUT_FILE = "../../test_data/pbmc.h5ad"
if not os.path.exists(OUTPUT_FILE):         
    adata = sc.read_10x_mtx(INPUT_PATH)
    adata.var_names_make_unique() 
    
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(
        adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
    )

    fake_cats = ["A", "B", "C"]
    fake_dogs = ["ALp", "Bet"]
    adata.obs["fake_cats"] = np.random.choice(fake_cats, adata.n_obs)
    adata.obs["fake_dogs"] = np.random.choice(fake_dogs, adata.n_obs)

    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=2000)

    adata.raw = adata
    adata = adata[:, adata.var.highly_variable]

    sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
    sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.leiden(adata, flavor="igraph", n_iterations=2)
    sc.tl.rank_genes_groups(adata, groupby="leiden", method="wilcoxon")
    adata.write("test_data/pbmc.h5ad")
else:
    adata = anndata.io.read_h5ad(OUTPUT_FILE)

## 2D Dimension Reduction Plot

The plotting functions can be run with few parameters. The following code plots 2D umap of the data colored by leiden clusters.

In [ ]:
from sclive.plotting.dimred_plt_2d_func import dimred_plt_2d
dimred_plt_2d(adata, "X_umap", "leiden")